In [3]:
import time
from call_gpt import *
from pprint import pprint
import glob

engine = "magic:1000080133:5a692de393b437bf98c6c3f36d273499"

In [2]:
import os

In [5]:
model_name = "llama2-7B-math-domain-normal-cot-0128"
data_name = "GSM8K"
result_dir = "/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/prediction"
ckpt_num = "checkpoint-60"
debug = False

output_dir = result_dir
model_name, data_name = model_name, data_name


# in data name I have checkpoint num
this_model_data_save_dir = os.path.join(output_dir, model_name, data_name)

ckpt_result_folder = os.path.join(this_model_data_save_dir, ckpt_num)
all_json_files = glob.glob(os.path.join(ckpt_result_folder, "*.json"))

In [9]:
all_current_tasks = []
for one_json_file in all_json_files:
    with open(one_json_file, "r") as f:
        one_json_content = f.readlines()
    one_json_content = [json.loads(x) for x in one_json_content]
    all_current_tasks.extend(one_json_content)

In [10]:
len(all_current_tasks)

1319

In [23]:
def load_prompts(path):
    with open(path, "r", encoding="utf8") as f:
        return "".join(f.readlines())


def process_files(prompt_dir, keep_examples, system_input_md_path):
    system_input = load_prompts(system_input_md_path)
    if len(keep_examples) == 0:
        keep_examples = [
            x for x in os.listdir(prompt_dir) if "example" in x and x[0] != "."
        ]

    example_files = []
    for fname in keep_examples:
        file_path = os.path.join(prompt_dir, fname)
        example_files.append(file_path)
        # api_list.extend(extract_actions_from_path(file_path))
    example_files = [x for x in example_files if "examples0.md" in x]
    examples_input = []
    for example_file in example_files:
        with open(example_file, "r") as f:
            example_lines = f.readlines()
        for example_line in example_lines:
            if example_line.startswith("USER"):
                example_line_user = example_line.strip("\n").replace("USER: ", "")
                examples_input.append({"role": "user", "content": example_line_user})
            elif example_line.startswith("ASSISTANT"):
                example_line_user = example_line.strip("\n").replace("ASSISTANT: ", "")
                examples_input.append(
                    {"role": "assistant", "content": example_line_user}
                )

    return system_input, examples_input


def build_user_query(question, analysis):
    analysis = analysis.split("\n\n\n")[0]
    return "USER:{}\n\n模型生成的解析: {}".format(question, analysis)


prompt_dir = "/mnt/pfs/zitao_team/tianqiaoliu/mathEval_data_check/matheval-lck/prompt"
system_input_info = "/mnt/pfs/zitao_team/tianqiaoliu/mathEval_data_check/matheval-lck/prompt/instruction.md"
system_input_global, examples_input_global = process_files(
    prompt_dir, [], system_input_info
)


def build_input_data(row):
    """
    根据给定的数据集行，构建输入数据。
    """
    # idx, row = row
    # query = row.to_json()
    row_content = row

    user_query = build_user_query(
        row_content["input"],
        row_content["response"],
    )
    # gpt4
    data = {
        "system": system_input_global,
        "examples": examples_input_global,
        "question": user_query,
        "temperature": 0,
        "frequency_penalty": 1,
        "presence_penalty": 1,
        "return_when_finished": False,
        "engine": "magic:1000080133:5a692de393b437bf98c6c3f36d273499",
        "max_tokens": 256,
        "max_retry": 20,
        "meta": row,
    }
    # # wenxin
    # data = {
    #     "message": query,
    # }
    # time.sleep(0.3)
    return data

In [24]:
input_data_gpt4 = []
for one_json_content in all_current_tasks:
    processed_data = build_input_data(one_json_content)
    input_data_gpt4.append(processed_data)

In [25]:
one_response = send_chat_request_async(**input_data_gpt4[0])

In [26]:
one_response

{'id': '3452c365-c106-4231-845f-e7bc8fb57631',
 '_id': 2032862,
 'created_at': '2024-01-31 14:34:10',
 'updated_at': '2024-01-31 14:34:10',
 'status': 1,
 'channel': 'default',
 'response': None,
 'request': {'body': {'messages': [{'role': 'system',
     'content': "# Instruction\n\n## You are the wise math word problem answer extractor:\n\n- You identify as math word problem answer extractor, **not** an assistant.\n- You will be provided an math word problem, the corresponding analysis for this math word problem from a generation model. You **should** understand the analysis and extract the answer from the disorganized analysis due to the analysis is from the generation model.\n- You can understand and communicate fluently in the problem's language of choice such as English, 中文, 日本語, Español, Français or Deutsch.\n- You **should** not solve the problem by yourself, you only job is to extract the answer from the given analysis.\n\n## On your profile and general capabilities:\n\n- Your 

In [27]:
with open("/mnt/pfs/zitao_team/tianqiaoliu/Project/teammates/hidden_cot/prediction/llama2-7B-math-domain-normal-cot-0128/GSM8K/checkpoint-60.json", "r") as f:
    gsm8k_ckpt_60 = json.load(f)

In [28]:
gsm8k_ckpt_60[0]

{'system': "# Instruction\n\n## You are the wise math word problem answer extractor:\n\n- You identify as math word problem answer extractor, **not** an assistant.\n- You will be provided an math word problem, the corresponding analysis for this math word problem from a generation model. You **should** understand the analysis and extract the answer from the disorganized analysis due to the analysis is from the generation model.\n- You can understand and communicate fluently in the problem's language of choice such as English, 中文, 日本語, Español, Français or Deutsch.\n- You **should** not solve the problem by yourself, you only job is to extract the answer from the given analysis.\n\n## On your profile and general capabilities:\n\n- Your responses should avoid being vague, controversial or off-topic.\n- Your logic and reasoning should be rigorous and intelligent.\n\n## On your output format:\n- You **should** ensure that the exracted answer aligns precisely with the format presented in th